In [ ]:
corpus = ['king is a strong man', 
          'queen is a  beautiful woman']

In [ ]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [ ]:
def remove_stop_words(corpus):
    stop_words = ['is', 'a']
    results = []
    for text in corpus:
        tmp = text.split(' ')
        for stop_word in stop_words:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))
    
    return results

In [ ]:
corpus = remove_stop_words(corpus)

In [ ]:
words = []
for text in corpus:
    for word in text.split(' '):
        words.append(word)

words = set(words)
words

{'', 'beautiful', 'king', 'man', 'queen', 'strong', 'woman'}

In [ ]:
word2int = {}

for i,word in enumerate(words):
    word2int[word] = i

sentences = []

for sentence in corpus:
    sentences.append(sentence.split())
    
WINDOW_SIZE = 2

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(sentence)) + 1] : 
            if neighbor != word:
                data.append([word, neighbor])

In [ ]:
import pandas as pd
for text in corpus:
    print(text)

df = pd.DataFrame(data, columns = ['input', 'label'])

df.head(10)

king strong man
queen  beautiful woman


,input,label
0,king,strong
1,king,man
2,strong,king
3,strong,man
4,man,king
5,man,strong
6,queen,beautiful
7,queen,woman
8,beautiful,queen
9,beautiful,woman


In [ ]:
word2int

{'': 0,
 'beautiful': 2,
 'king': 3,
 'man': 6,
 'queen': 1,
 'strong': 5,
 'woman': 4}

# Define Tensorflow Graph

In [ ]:
import tensorflow as tf
import numpy as np

ONE_HOT_DIM = len(words)

# function to convert numbers to one hot vectors
def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding

X = [] # input word
Y = [] # target word

for x, y in zip(df['input'], df['label']):
    X.append(to_one_hot_encoding(word2int[ x ]))
    Y.append(to_one_hot_encoding(word2int[ y ]))

# convert them to numpy arrays
X_train = np.asarray(X)
Y_train = np.asarray(Y)

# making placeholders for X_train and Y_train
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

# word embedding will be 2 dimension for 2d visualization
EMBEDDING_DIM = 2 

# hidden layer: which represents word vector eventually
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1])) #bias
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function: cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

# training operation
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

In [ ]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 27kB/s 
     |████████████████████████████████| 3.8MB 31.2MB/s 
     |████████████████████████████████| 512kB 38.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=016c3b9bbad5f99ac91cd13cf59ba7939658f59d6dae4e307fdd7d9eb4fd0995
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: 

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

iteration = 30000
for i in range(iteration):
    # input is X_train which is one hot encoded word
    # label is Y_train which is one hot encoded neighbor word
    sess.run(train_op, feed_dict={x: X_train, y_label: Y_train})
    if i % 3000 == 0:
        print('iteration '+str(i)+' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))

iteration 0 loss is :  2.7752602
iteration 3000 loss is :  0.91535395
iteration 6000 loss is :  0.8927775
iteration 9000 loss is :  0.87948054
iteration 12000 loss is :  0.87096715
iteration 15000 loss is :  0.8649745
iteration 18000 loss is :  0.8604762
iteration 21000 loss is :  0.8569436
iteration 24000 loss is :  0.85407585
iteration 27000 loss is :  0.8516879


In [ ]:
# Now the hidden layer (W1 + b1) is actually the word look up table
vectors = sess.run(W1 + b1)
print(vectors)

[[ 0.3042987  -0.5721961 ]
 [ 3.2355022  -1.1452657 ]
 [ 0.4157862  -7.2609396 ]
 [ 2.2080314   2.9899757 ]
 [-0.10864949 -0.6752644 ]
 [-3.7633772   6.6740255 ]
 [-0.46691036  0.5139273 ]]


In [ ]:
w2v_df = pd.DataFrame(vectors, columns = ['x1', 'x2'])
w2v_df['word'] = words
w2v_df = w2v_df[['word', 'x1', 'x2']]
w2v_df

,word,x1,x2
0,,0.304299,-0.572196
1,queen,3.235502,-1.145266
2,beautiful,0.415786,-7.260940
3,king,2.208031,2.989976
4,woman,-0.108649,-0.675264
5,strong,-3.763377,6.674026
6,man,-0.466910,0.513927
